# VacationPy

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from api_keys import g_key

# Store Part I results into DataFrame

In [2]:
weather_data_toload = ('D:/GT-VIRT-DATA-PT-03-2022-U-LOL/Python_api_challenge/WeatherPy/weather_df.csv')
weather_data = pd.read_csv(weather_data_toload)
weather_data_drop = weather_data.dropna()
del weather_data_drop["Unnamed: 0"]
weather_data_drop.head()

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,tasiilaq,GL,2022-04-23 07:20:53,65.6145,-37.6368,100.0,95.0,45.032,1.01
1,sao joao da barra,BR,2022-04-23 07:20:54,-21.6403,-41.0511,52.0,63.0,82.760,3.56
2,hambantota,LK,2022-04-23 07:20:54,6.1241,81.1185,90.0,76.0,84.164,4.68
3,upernavik,GL,2022-04-23 07:20:54,72.7868,-56.1549,100.0,96.0,6.584,4.02
4,ushuaia,AR,2022-04-23 07:20:55,-54.8000,-68.3000,40.0,75.0,42.728,3.09


# Humidity Heatmap

In [3]:
locations= weather_data_drop[["Latitude", "Longitude"]]
weights = weather_data_drop['Humidity (%)'].astype(float)

fig= gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,weights= weights, dissipating = False, max_intensity = 100, point_radius = 2 )

fig.add_layer(heat_layer)

plt.savefig("Humidity Heatmap")

fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

# Create new DataFrame fitting weather criteria

In [4]:
location_temp_max = weather_data_drop[weather_data_drop['Max Temp (F)']<= 90]
location_temp_min = location_temp_max[location_temp_max['Max Temp (F)']>=75]

location_wind = location_temp_min[location_temp_min['Wind Speed (mph)']< 15]
location_cloudiness = location_wind[location_wind['Cloudiness (%)'] == 0]
location_humidity=location_cloudiness[location_cloudiness['Humidity (%)']< 70]
location_humidity_index = location_humidity.reset_index()

del location_humidity_index["index"]
location_humidity_index


,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,arraial do cabo,BR,2022-04-23 07:20:58,-22.9661,-42.0278,0.0,58.0,84.434,6.17
1,santa luzia,BR,2022-04-23 07:21:06,-19.7697,-43.8514,0.0,55.0,79.214,2.06
2,vila velha,BR,2022-04-23 07:20:56,-20.3297,-40.2925,0.0,46.0,88.016,5.14
3,genc,TR,2022-04-23 07:22:45,38.7507,40.5603,0.0,19.0,77.630,4.12
4,atar,MR,2022-04-23 07:22:53,20.5169,-13.0499,0.0,24.0,82.724,5.14
5,ankara,TR,2022-04-23 07:20:00,39.9199,32.8543,0.0,23.0,76.820,3.09
6,haifa,IL,2022-04-23 07:23:09,32.8156,34.9892,0.0,60.0,75.434,4.63
7,terrasini,IT,2022-04-23 07:23:25,38.1460,13.0844,0.0,62.0,76.172,4.12


# Hotel Map

In [5]:
ideal_locations = location_humidity_index[['Latitude', 'Longitude']]
ideal_weights = location_humidity_index['Humidity (%)'].astype(float)

fig=gmaps.figure()
ideal_heat_layer = gmaps.heatmap_layer(locations = ideal_locations,weights= ideal_weights, dissipating = False, max_intensity = 100, point_radius = 2 )
fig.add_layer(ideal_heat_layer)

plt.savefig("Humidity Heatmap_Ideal Locations")
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [6]:
Hotel = []

for city in range(len(location_humidity_index["City"])):
    lat = location_humidity_index.loc[city]['Latitude']
    lng= location_humidity_index.loc[city]['Longitude']
    coords= f'{lat}, {lng}'
    
    params = { 
        "location": coords,
        "types": "lodging",
        "radius": 5000,
        "key": g_key   }
 
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params).json()
    try:
        Hotel.append(response['results'][0]['name'])
    except: 
        Hotel.append("No found any hotel")
location_humidity_index["The Best Hotels"] = Hotel
location_humidity_index


,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),The Best Hotels
0,arraial do cabo,BR,2022-04-23 07:20:58,-22.9661,-42.0278,0.0,58.0,84.434,6.17,Pousada Porto Praia
1,santa luzia,BR,2022-04-23 07:21:06,-19.7697,-43.8514,0.0,55.0,79.214,2.06,Pousada Chao Earth
2,vila velha,BR,2022-04-23 07:20:56,-20.3297,-40.2925,0.0,46.0,88.016,5.14,Hotel Vitória Palace
3,genc,TR,2022-04-23 07:22:45,38.7507,40.5603,0.0,19.0,77.630,4.12,Genc Ogretmenevi
4,atar,MR,2022-04-23 07:22:53,20.5169,-13.0499,0.0,24.0,82.724,5.14,Oumou Elghoura
5,ankara,TR,2022-04-23 07:20:00,39.9199,32.8543,0.0,23.0,76.820,3.09,Ramada by Wyndham Ankara
6,haifa,IL,2022-04-23 07:23:09,32.8156,34.9892,0.0,60.0,75.434,4.63,Dan Panorama Haifa Hotel
7,terrasini,IT,2022-04-23 07:23:25,38.1460,13.0844,0.0,62.0,76.172,4.12,Florio Park Hotel


# The best hotel map with Info Box

In [7]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{The Best Hotels}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in location_humidity_index.iterrows()]
hotel_locations = location_humidity_index[["Latitude", "Longitude"]]

markers = gmaps.marker_layer(locations = hotel_locations, info_box_content=[f'The Best Hotels: {Hotel}' for Hotel in Hotel])
fig.add_layer(markers)

plt.savefig("The Best Hotels.png")
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>